In [ ]:
# Modify this based on your system
SAMPLES_DATA_DIR = 'C:/opencv/sources/samples/data/'

In [ ]:
# Harris Corner Detector
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the image (load as colored)
img = cv2.imread(SAMPLES_DATA_DIR+'chessboard.png',cv2.IMREAD_COLOR)
# Create a gray-scaled version
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect Harris corners
# cv2.cornerHarris(src, neighborhoodSize, sobelKernelSize, Harris_k)
# corners is the Harris corner map, a cornerness for each pixel
corners = cv2.cornerHarris(img_gray,2,3,0.04)
# Normalize the Harris cornermap between 0 and 1
cv2.normalize(corners,corners,0,1,cv2.NORM_MINMAX)

# Threshold for an optimal value, it may vary depending on the image.
threshold = 0.6

# list of all points, higher than the threshold
# loc[0] is an numpy array, containing the x coordinates
# loc[1] is an numpy array, containing the y coordinates
loc = np.where( corners >= threshold)

# loop through the points
# zip() aggregates elements from loc[0] and loc[1]
for pt in zip(*loc[::-1]):  
    # draw a filled circle on each point
    cv2.circle(img, pt,50, (255,0,0), -1)

# Display
plt.imshow(img),plt.xticks([]), plt.yticks([]),plt.show()

In [ ]:
# Detour on zip
x=[1,2,3]
y=[4,5,6]
for pt in zip(x,y):
    print(pt)

In [ ]:
# SIFT Feature Detector
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the image 
img = cv2.imread(SAMPLES_DATA_DIR+'lena.jpg')

# Create a SIFT object
sift = cv2.xfeatures2d.SIFT_create()

# Detect SIFT features, with no masks
keypoints = sift.detect(img,None)

# Draw the keypoints
img_disp = img.copy()
cv2.drawKeypoints(img,keypoints,img_disp,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Display
cv2.imshow("img",img_disp)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Num of SIFT keypoints
print('Num keypoints: '+ str(len(keypoints)))

In [ ]:
# SIFT Descriptors
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the image 
img = cv2.imread(SAMPLES_DATA_DIR+'lena.jpg')

# Create a SIFT object
sift = cv2.xfeatures2d.SIFT_create()

# Detect SIFT features, with no masks
keypoints = sift.detect(img,None)

# Compute SIFT descriptors
keypoints,descriptors = sift.compute(img,keypoints)

# Alternatively, we can use keypoints,descriptors = sift.detectAndCompute(gray,None) to get both

# Num of SIFT keypoints/descriptors
print('descriptors size: '+ str(descriptors.shape))

# Plot a couple of 1x128 SIFT descriptors using matplotlib plot()
Num = 3
for i in range(Num):
    # Plot the 256x1 np array using matplotlib plot()
    plt.plot(descriptors[i])
    plt.xlim([0,128]) # Set x-axis limit
plt.show()

In [ ]:
# Feature matching
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the image 
img1 = cv2.imread(SAMPLES_DATA_DIR+'lena.jpg')
height, width = img1.shape[0:2] # image size

# Create a rotated, scaled duplicate
# rotation marix around the center pixel, 30 degrees,scale of 1.2
M = cv2.getRotationMatrix2D((width/2,height/2),30,1.2)
# Apply the transformation matrix
img2 = cv2.warpAffine(img1,M,(width,height))

# Create a SIFT object
sift = cv2.xfeatures2d.SIFT_create()

# Compute the keypoints/descriptors
keypoints1,descriptors1 = sift.detectAndCompute(img1,None)
keypoints2,descriptors2 = sift.detectAndCompute(img2,None)

# Matching
# use cv2.BFMatcher for the assignment
# we will try to match a randomly-selected feature from img1 to img2
idx = 0
# Loop through all descriptors in img2 to find the closest match
bestMatch_idx = -1
bestMatch_dist = float('inf')
for i in range(descriptors2.shape[0]):
    dist = cv2.norm(descriptors1[idx],descriptors2[i],cv2.NORM_L2)
    if dist < bestMatch_dist:
        bestMatch_dist = dist
        bestMatch_idx = i
        
# Draw the keypoints
img_disp1 = img1.copy()
img_disp2 = img2.copy()
# draw a filled circle on each point
center1 = (int(keypoints1[idx].pt[0]),int(keypoints1[idx].pt[1]))
center2 = (int(keypoints2[bestMatch_idx].pt[0]),int(keypoints2[bestMatch_idx].pt[1]))
cv2.circle(img_disp1, center1, 10, (255,0,0), -1)
cv2.circle(img_disp2, center2, 12, (255,0,0), -1)

# Display
plt.subplot(221),plt.imshow(img_disp1),plt.title('img1')
plt.xticks([]), plt.yticks([])
plt.subplot(222),plt.imshow(img_disp2),plt.title('img2')
plt.xticks([]), plt.yticks([])

# Plot the descriptors
plt.subplot(223),plt.plot(descriptors1[idx]),plt.title('desc1')
plt.xlim([0,128]) # Set x-axis limit
plt.subplot(224),plt.plot(descriptors2[bestMatch_idx]),plt.title('desc2')
plt.xlim([0,128]) # Set x-axis limit
plt.show() # display all subplots